In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import tkinter
import tkinter.filedialog
from PIL import Image,ImageTk
from torchvision import transforms as transforms
import os
import skimage.io as io
from skimage import data_dir
from sys import exit

In [2]:
#实现在本地电脑选择图片
def choose_file():
    global select_file
    select_file = tkinter.filedialog.askopenfilename(title='选择图片')
    e.set(select_file)
    #打开图片
    load = Image.open(select_file)
    load = transforms.Resize((300,400))(load)
    #加载显示图片
    render = ImageTk.PhotoImage(load)
    img  = tkinter.Label(win,image=render)
    img.image = render
    #设置图像的显示位置
    img.place(x=50,y=100)

In [3]:
def NumberOCR():
    
    def cvShow(name,img):
        cv2.imshow(f'{name}',img)
        cv2.waitKey()
        cv2.destroyAllWindows()
    
    def DelNoise(contours,value):
        #由于元组不可删，将轮廓元组转化为列表
        CtoL=list(contours)
        area=np.zeros(len(contours))
        i=0
        while(i<len(CtoL)):
            area[i]=cv2.contourArea(CtoL[i])
            if area[i]<value:
                del CtoL[i]
            else:
                i=i+1
        return CtoL
    
    temp=original
    img=cv2.imread(select_file)
    img1=np.copy(img)
    
    #对于不同的图像选择使用高斯滤波，原图高分辨率可用，低分辨率不可用
    img1=cv2.GaussianBlur(img1,(5,5),0,0)
    #cvShow('gaus',img1)
    #对原图像进行一个滤波操作模糊噪点，保留边缘信息
    img1=cv2.bilateralFilter(img1, 15, 75,100)
    imgG=cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)
    imgG1=np.copy(imgG)
    img1=np.copy(img)
    #阈值分割，获取轮廓
    threshAm=cv2.adaptiveThreshold(imgG,255,cv2.ADAPTIVE_THRESH_MEAN_C,
                                  cv2.THRESH_BINARY_INV,101,5)
    contours,hierarchy=cv2.findContours(threshAm,cv2.RETR_TREE,
                                        cv2.CHAIN_APPROX_SIMPLE)
    CtoL=DelNoise(contours,20)
    
    #轮廓近似，epsilon为准确度参数，epsilon的值越小，多边形逼近更准确
    approx=[0]*len(CtoL)
    epsilon=11
    for i in range(len(CtoL)):
        #第一个参数是要逼近的轮廓，第二个参数是准确值，第三个参数是设定弧线是否闭合
        approx[i]=cv2.approxPolyDP(CtoL[i],epsilon,True)
    area=np.zeros(len(approx))
    
    #对轮廓面积从大到小进行一个排序
    for i in range(len(approx)):
        area[i]=cv2.contourArea(approx[i])
    area=sorted(area,reverse=True)
    sort=[0]*len(approx)
    j=0
    while(j<len(area)):
        i=0
        for i in range(len(approx)):
            if cv2.contourArea(approx[i])==area[j]:
                sort[j]=i
                break
        j=j+1
        
    #按照轮廓面积从小到大遍历，取最大的四边形轮廓作为图像ROI
    cnt=np.array([])
    for i in range(len(sort)):
        if len(approx[sort[i]])==4:
            cnt=approx[sort[i]]
            break
            
    #通过四边形轮廓定位四个坐标进行ROI
    x1=min(cnt[0][0][0],cnt[1][0][0])
    x2=max(cnt[2][0][0],cnt[3][0][0])
    y1=min(cnt[0][0][1],cnt[3][0][1])
    y2=max(cnt[1][0][1],cnt[2][0][1])
    print(x1,x2,y1,y2)
    #图像ROI
    roi=img1[y1:y2,x1:x2]
    #统一大小
    roi=cv2.resize(roi,(500,125))
    
    #转灰度
    roi1=np.copy(roi)
    roiG=cv2.cvtColor(roi,cv2.COLOR_BGR2GRAY)
    
    #ROI之后进行一次图像增强，此处为局部直方图均衡化
    clache=cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    roiG=clache.apply(roiG)
    
    #使用阈值分割
    ret,thresh=cv2.threshold(roiG,155,255,cv2.THRESH_BINARY_INV)
    contours,hierarchy=cv2.findContours(thresh,cv2.RETR_TREE,
                                    cv2.CHAIN_APPROX_SIMPLE)
    
    #去除噪声轮廓
    CtoL=DelNoise(contours,200)
    
    #获取轮廓的外接四边形用于数字ROI
    bounding_boxes=[cv2.boundingRect(c) for c in CtoL]
    i=0
    while i<(len(bounding_boxes)):
        [x,y,w,h]=bounding_boxes[i]
        #先排除长宽比不符合数字特征的外接矩形
        if w>h:
            del bounding_boxes[i]
        else:
            i=i+1
    
    #删除在其他外接矩形内部的矩形
    i=0
    while i<(len(bounding_boxes)):
        [xi,yi,wi,hi]=bounding_boxes[i]
        j=0
        while j<(len(bounding_boxes)):
            [xj,yj,wj,hj]=bounding_boxes[j]
            if xi<=xj and xi+wi>xj+wj and yi<=yj and yi+hi>yj+hj:
                del bounding_boxes[j]
            else:
                j=j+1
        i=i+1
    print(bounding_boxes)
    
    #删除面积太小的轮廓
    i=0
    while i<(len(bounding_boxes)):
        [xi,yi,wi,hi]=bounding_boxes[i]
        square=wi*hi
        if square<800:
            del bounding_boxes[i]
        else:
            i=i+1
    
    #实现数字ROI
    #将数字的外接矩形从左到右排序
    bounding_boxes=sorted(bounding_boxes)

    #ROI之后进行一次图像增强
    clache=cv2.createCLAHE(clipLimit=2.0,tileGridSize=(8,8))
    #用列表将数字ROI图片存起来
    num=[0]*len(bounding_boxes)
    for i in range(len(bounding_boxes)):
        [x,y,w,h]=bounding_boxes[i]
        #图像ROI
        num[i]=roi1[y:y+h,x:x+w]
        #统一大小
        num[i]=cv2.resize(num[i],(50,100))
        #转灰度
        num[i]=cv2.cvtColor(num[i],cv2.COLOR_BGR2GRAY)
        #cvShow('numGray',num[i])
        #ROI之后进行一次图像增强，此处为局部直方图均衡化
        num[i]=clache.apply(num[i])
        #cvShow('His',num[i])
        #二值化
        ret,num[i]=cv2.threshold(num[i],120,255,cv2.THRESH_BINARY)
        #cvShow('numthre',num[i])
    
    #读入训练数据
    #预先初始化分配内存，解决赋值内存空间混乱问题
    train_data=[0]*10
    for i in range(10):
        train_data[i]=[0 for i in range(10)]

    coll=[0]*10

    root_dir=os.listdir("F:/ComputerVision/Class/shiyan5/digitaldetection/data")
    for i in range(len(root_dir)):
        pathname=os.path.join("F:/ComputerVision/Class/shiyan5/digitaldetection/data",root_dir[i])
        str=pathname+ '/*.bmp'
        coll[i]=io.ImageCollection(str)
        #统一训练数据和测试数据的大小
        for j in range(len(coll[i])):
            train_data[i][j]=cv2.resize(coll[i][j],(num[0].shape[1],num[0].shape[0]))

    #利用读入的训练数据构造数据集
    trainData=np.zeros((100,len(train_data[0][0].ravel())))
    #共有100个训练数据，每个数字10个，将每个数字图像的二维矩阵转化为一维行向量
    for i in range(len(train_data)):
        for j in range(len(train_data[i])):
            trainData[i*10+j]=train_data[i][j].ravel()
    trainData=np.float32(trainData)
    #获得100(数据图像总数)个行向量，每个行向量有像素点总数个元素
    
    #构造训练集标签
    labels=np.zeros((100,1))
    j=0
    for i in range(len(labels)):
        if int(i/10)>j:
            j=j+1
        labels[i]=j
    labels=np.float32(labels)
    
    #利用之前获取的数字ROI图片构造预测集
    predictData=np.zeros((len(num),len(train_data[0][0].ravel())))
    for i in range(len(num)):
        predictData[i]=num[i].ravel()
    predictData=np.float32(predictData)

    #使用knn分类器分类
    knn=cv2.ml.KNearest_create()
    knn.train(trainData,cv2.ml.ROW_SAMPLE,labels)
    ret,results,neighbours,dist=knn.findNearest(predictData,5)
    
    string=""
    for i in range(len(results)):
        string=string+'%d'%(results[i][0])
    img1=cv2.putText(img1,string,(10,50),cv2.FONT_HERSHEY_SIMPLEX,2,(0,255,0),2)
    
    new_im=Image.fromarray(img1)
    new_im = transforms.Resize((300,400))(new_im)
    render = ImageTk.PhotoImage(new_im)
    global img2
    img2.destroy()
    img2  = tkinter.Label(image=render)
    img2.image = render
    img2.place(x=550,y=100)

In [4]:
#创建一个界面窗口
win = tkinter.Tk()
win.title("picture process")
win.geometry("1080x680")
#设置全局变量
original = Image.new('RGB', (300, 400))
save_img = Image.new('RGB', (300, 400))
count =0
img2 = tkinter.Label(win)

In [5]:
#显示路径
e = tkinter.StringVar()
e_entry = tkinter.Entry(win, width=68, textvariable=e)
e_entry.pack()

In [6]:
#设置选择图片的按钮
button1 = tkinter.Button(win, text ="  Select  ", command = choose_file)
button1.place(x=200,y=500)

In [7]:
#设置图片OCR的按钮
button2=tkinter.Button(win,text="  OCR  ",command=NumberOCR)
button2.place(x=700,y=500)

In [8]:
#设置标签分别为原图像和修改后的图像
label1 = tkinter.Label(win,text="Original Picture")
label1.place(x=200,y=50)

label2 = tkinter.Label(win,text="Result Picture")
label2.place(x=700,y=50)

In [ ]:
#设置退出按钮
button0 = tkinter.Button(win,text="  Exit  ",command=win.quit)
button0.place(x=1000,y=500)
win.mainloop()

31 346 152 219
[(350, 43, 28, 51), (280, 43, 17, 49), (202, 35, 31, 56), (419, 34, 28, 45), (123, 31, 30, 58), (41, 29, 30, 57)]
